# 0. Setup

Create an MLRun project and the streams relevant for our scenario.

![Model deployment with streaming Real-time operational Pipeline](../../assets/images/model-deployment-with-streaming.png)

### Load configurations

In [1]:
import v3io.dataplane
from mlrun import new_project, mlconf

In [2]:
%run config.py

## Create V3IO Client

In [3]:
v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=V3IO_ACCESS_KEY)

## Manage Streams

#### Delete all streams

In [4]:
for stream_name, stream_config in STREAM_CONFIGS.items():
    resp = v3io_client.delete_stream(container=CONTAINER, path=stream_config['path'], 
                                     raise_for_status=v3io.dataplane.RaiseForStatus.never)
    print(f'Delete Stream call for stream {stream_name} returned with status {resp.status_code}, and content: {resp.body.decode("utf-8")}')

Delete Stream call for stream generated-stream returned with status 204, and content: 
Delete Stream call for stream incoming-events-stream returned with status 204, and content: 
Delete Stream call for stream enriched-events-stream returned with status 204, and content: 
Delete Stream call for stream serving-stream returned with status 204, and content: 
Delete Stream call for stream inference-stream returned with status 204, and content: 


#### Create all streams

In [5]:
for stream_name, stream_config in STREAM_CONFIGS.items():
    print(stream_config['path'])
    resp = v3io_client.create_stream(container=CONTAINER,
                                     path=stream_config['path'],
                                     shard_count=stream_config['shard_count'],
                                    raise_for_status=v3io.dataplane.RaiseForStatus.never)
    print(f'Create Stream call for stream {stream_name} returned with status {resp.status_code}, and content: {resp.body.decode("utf-8")}')

iguazio/examples/model-deployment-with-streaming/data/generated-stream
Create Stream call for stream generated-stream returned with status 204, and content: 
iguazio/examples/model-deployment-with-streaming/data/incoming-events-stream
Create Stream call for stream incoming-events-stream returned with status 204, and content: 
iguazio/examples/model-deployment-with-streaming/data/enriched-events-stream
Create Stream call for stream enriched-events-stream returned with status 204, and content: 
iguazio/examples/model-deployment-with-streaming/data/serving-stream
Create Stream call for stream serving-stream returned with status 204, and content: 
iguazio/examples/model-deployment-with-streaming/data/inference-stream
Create Stream call for stream inference-stream returned with status 204, and content: 
